In [2]:
import urllib
from lxml import html
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
import itertools
import StringIO
import matplotlib
import matplotlib.pyplot as plt
import subprocess
import os
%matplotlib inline

In [4]:
df = pd.read_csv('debate.csv')
df.head(n=5)

,Line,Speaker,Text,Date
0,1,Holt,Good evening from Hofstra University in Hempst...,9/26/16
1,2,Audience,(APPLAUSE),9/26/16
2,3,Clinton,"How are you, Donald?",9/26/16
3,4,Audience,(APPLAUSE),9/26/16
4,5,Holt,Good luck to you.,9/26/16


In [130]:
page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119181').read()
soup = BeautifulSoup(page, "lxml")
#speech = soup.find_all('span')[6]
speech = soup.find_all('p')
for sentence in speech:
    sentence = str(sentence)[3:-4]


In [156]:
page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119161').read()
soup = BeautifulSoup(page, "lxml")
#speech = soup.find_all('span')[6]
soup.find_all('p')

[<p>Before I begin with my remarks, I do want to say how proud I am of our brave first responders working to keep us safe after the attacks of the last weekend in New York, New Jersey and Minnesota. There are now reports of a suspect in custody, but we must remain vigilant. This is a fast-moving situation and a sobering reminder that we need steady leadership in a dangerous world.</p>,
 <p>I'm here to talk about a number of the issues that are part of this election but really, much more than that, they are part of our future \u2013 the kind of country we want to have, the kind of people we want to be and, particularly, what kind of opportunities we should be providing for the young people of America. I have a proud Owl on my staff, Jamira Burley. A Philadelphia native who became an activist working to end the epidemic of gun violence right here in Philadelphia. She loves Temple and we love her. And I also want to thank Lauren for that introduction.</p>,
 <p>Jamira and Lauren are two ex

In [206]:
valid_ids = range(161, 199)
for i in range(200,210):
    valid_ids.append(i)
trump_speeches = []
for valid_id in valid_ids:
    page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119' + str(valid_id)).read()
    soup = BeautifulSoup(page, "lxml")
    #speech = soup.find_all('span')[6]
    speech = soup.find_all('p')
    for sentence in speech:
        sentence = str(sentence)[3:-4]
        trump_speeches.append(sentence)

In [207]:
print len(trump_speeches)

3757


In [3]:
valid_ids = range(148, 165)
for i in range(497,503):
    valid_ids.append(i)
for i in range(688,702):
    valid_ids.append(i)
clinton_speeches = []
for valid_id in valid_ids:
    page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119' + str(valid_id)).read()
    soup = BeautifulSoup(page, "lxml")
    #speech = soup.find_all('span')[6]
    speech = soup.find_all('p')
    for sentence in speech:
        sentence = str(sentence)[3:-4]
        clinton_speeches.append(sentence)

In [6]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#fp = open("test.txt")
#data = fp.read()



In [7]:
trump_df = pd.DataFrame({}, columns = df.columns)
clinton_df = pd.DataFrame({}, columns = df.columns)
trump = []
clinton = []

end_t = 0
end_c = 0
for i in range(df.shape[0]):
#for i in range(5):
    if df['Speaker'].values[i] == 'Trump':
        speech = df['Text'].values[i]
        sentences = tokenizer.tokenize(speech)
        for sentence in sentences:
            trump_df.loc[end_t] = [end_t, df['Speaker'].values[i], sentence, df['Date'].values[i]]
            trump.append(sentence)
            end_t += 1
    if df['Speaker'].values[i] == 'Clinton':
        speech = df['Text'].values[i]
        sentences = tokenizer.tokenize(speech)
        for sentence in sentences:
            clinton_df.loc[end_c] = [end_c, df['Speaker'].values[i], sentence, df['Date'].values[i]]
            clinton.append(sentence)
            end_c += 1
            
        
        
    

In [202]:
print len(trump)

2020


In [209]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
for i in trump_speeches:
    trump.append(i)


In [8]:
for i in clinton_speeches:
    clinton.append(i)

In [9]:
print len(clinton)

2833


In [11]:
name = 'astro_sentences.txt'
f = open(name,'w')
for sentence in clinton:
	f.write(sentence + ',1\n')
for sentence in trump:
    f.write(sentence+',0\n')
f.close()

In [210]:
print trump[:10]

['Thank you, Lester.', 'Our jobs are fleeing the country.', "They're going to Mexico.", "They're going to many other countries.", 'You look at what China is doing to our country in terms of making our product.', "They're devaluing their currency, and there's nobody in our government to fight them.", 'And we have a very good fight.', 'And we have a winning fight.', "Because they're using our country as a piggy bank to rebuild China, and many other countries are doing the same thing.", "So we're losing our good jobs, so many of them."]


In [212]:
name = 'trump_sentences.txt'
f = open(name,'w')
for sentence in trump:
	f.write(sentence + ' ')
f.close()

In [204]:

trump = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in trump]

trump_words = [nltk.word_tokenize(sent) for sent in trump]
#clinton_words = [nltk.word_tokenize(sent) for sent in clinton]
print type(trump)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 5: ordinal not in range(128)

In [ ]:
word_freq = nltk.FreqDist(itertools.chain(*trump_words))

In [197]:
print type(word_freq)
print len(word_freq)
word_freq

<class 'nltk.probability.FreqDist'>
48


FreqDist({u' ': 11554,
          u'"': 9,
          u'$': 1,
          u'-': 8,
          u'.': 22,
          u'1': 4,
          u'2': 5,
          u'3': 2,
          u'4': 13,
          u'5': 6,
          u'6': 2,
          u'7': 2,
          u'9': 7,
          u'<': 2,
          u'A': 5777,
          u'C': 11554,
          u'D': 5777,
          u'E': 40439,
          u'N': 28885,
          u'R': 5777,
          u'S': 17331,
          u'T': 23108,
          u'[': 1,
          u'_': 11554,
          u'a': 729,
          u'b': 290,
          u'c': 65,
          u'd': 21,
          u'e': 63,
          u'f': 92,
          u'g': 29,
          u'h': 277,
          u'i': 981,
          u'j': 51,
          u'k': 2,
          u'l': 87,
          u'm': 135,
          u'n': 181,
          u'o': 258,
          u'p': 51,
          u'q': 5,
          u'r': 38,
          u's': 321,
          u't': 891,
          u'u': 19,
          u'v': 12,
          u'w': 900,
          u'y': 195})

In [196]:
vocab_size = 2200
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Find most frequently used words
vocab = word_freq.most_common(vocab_size-1)

# Get an index_to_word vector and a word_to_index vector
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [90]:
# Substitute unknown token for words not in vocab
for i, sent in enumerate(trump_words):
    trump_words[i] = [w if w in word_to_index else unknown_token for w in sent]

In [91]:
# Training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in trump_words])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in trump_words])

In [92]:
X_train[:10]

array([[1, 806, 8, 3, 290, 2], [1, 224, 126, 28, 1633, 4, 40, 2],
       [1, 61, 22, 26, 6, 376, 2], [1, 61, 22, 26, 6, 73, 93, 171, 2],
       [1, 53, 55, 36, 33, 242, 21, 88, 6, 32, 40, 17, 394, 7, 504, 32, 833, 2],
       [1, 61, 22, 1624, 117, 1392, 3, 9, 110, 13, 284, 17, 32, 537, 6, 312, 59, 2],
       [1, 15, 16, 14, 10, 29, 135, 312, 2],
       [1, 15, 16, 14, 10, 948, 312, 2],
       [1, 205, 20, 22, 2199, 32, 40, 74, 10, 1475, 1413, 6, 1690, 242, 3, 9, 73, 93, 171, 28, 88, 4, 316, 108, 2],
       [1, 81, 16, 22, 485, 32, 135, 126, 3, 46, 73, 7, 59, 2]], dtype=object)

In [93]:
y_train[:10]

array([[806, 8, 3, 290, 2, 0], [224, 126, 28, 1633, 4, 40, 2, 0],
       [61, 22, 26, 6, 376, 2, 0], [61, 22, 26, 6, 73, 93, 171, 2, 0],
       [53, 55, 36, 33, 242, 21, 88, 6, 32, 40, 17, 394, 7, 504, 32, 833, 2, 0],
       [61, 22, 1624, 117, 1392, 3, 9, 110, 13, 284, 17, 32, 537, 6, 312, 59, 2, 0],
       [15, 16, 14, 10, 29, 135, 312, 2, 0],
       [15, 16, 14, 10, 948, 312, 2, 0],
       [205, 20, 22, 2199, 32, 40, 74, 10, 1475, 1413, 6, 1690, 242, 3, 9, 73, 93, 171, 28, 88, 4, 316, 108, 2, 0],
       [81, 16, 22, 485, 32, 135, 126, 3, 46, 73, 7, 59, 2, 0]], dtype=object)

In [94]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [95]:
class RNNNumpy:
     
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))


In [96]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]
 
RNNNumpy.forward_propagation = forward_propagation

In [97]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)
 
RNNNumpy.predict = predict

In [98]:
np.random.seed(10)
model = RNNNumpy(vocab_size)
o, s = model.forward_propagation(X_train[10])
print o.shape
print o

(28, 2200)
[[ 0.00045788  0.00046038  0.00045607 ...,  0.00045893  0.00045034
   0.00045592]
 [ 0.00045144  0.00045402  0.00046258 ...,  0.00045643  0.00046381
   0.00045718]
 [ 0.00046353  0.00045254  0.00045613 ...,  0.00045554  0.00045153
   0.00044744]
 ..., 
 [ 0.00045214  0.00045761  0.00045249 ...,  0.00045004  0.00045702
   0.0004534 ]
 [ 0.00045532  0.00045119  0.00045408 ...,  0.00045495  0.0004529
   0.00045495]
 [ 0.00045195  0.00045516  0.0004519  ...,  0.00045275  0.00046396
   0.00045217]]


In [99]:
predictions = model.predict(X_train[10])
print predictions.shape
print predictions

(28,)
[ 964 2032 1903  568 1792 1687 1746  241  714 1694  801  846  513  557  991
  614 1682 1039  972 1652 1861  671  267 2040 1508 1760 2135 1002]


In [100]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L
 
def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N
 
RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [101]:
# Limit to 1000 examples to save time
print "Expected Loss for random predictions: %f" % np.log(vocab_size)
print "Actual loss: %f" % model.calculate_loss(X_train, y_train)

Expected Loss for random predictions: 7.696213
Actual loss: 7.696676


In [102]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNNNumpy.bptt = bptt

In [103]:
import operator
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = self.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = self.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = self.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
 
RNNNumpy.gradient_check = gradient_check
 
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])


Performing gradient check for parameter U with size 1000.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 1000.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [104]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
 
RNNNumpy.sgd_step = numpy_sdg_step


In [105]:
import datetime
import sys
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5 
                print "Setting learning rate to %f" % learning_rate
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [106]:
np.random.seed(10)
model = RNNNumpy(vocab_size)
%timeit model.sgd_step(X_train[10], y_train[10], 0.005)

The slowest run took 4.98 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 40.2 ms per loop


In [107]:


np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocab_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)


2016-12-08 22:36:39: Loss after num_examples_seen=0 epoch=0: 7.696246
2016-12-08 22:36:43: Loss after num_examples_seen=100 epoch=1: 7.678721
2016-12-08 22:36:45: Loss after num_examples_seen=200 epoch=2: 7.649209
2016-12-08 22:36:47: Loss after num_examples_seen=300 epoch=3: 7.584138
2016-12-08 22:36:49: Loss after num_examples_seen=400 epoch=4: 7.426806
2016-12-08 22:36:51: Loss after num_examples_seen=500 epoch=5: 5.984401
2016-12-08 22:36:54: Loss after num_examples_seen=600 epoch=6: 5.446160
2016-12-08 22:36:56: Loss after num_examples_seen=700 epoch=7: 5.240658
2016-12-08 22:36:58: Loss after num_examples_seen=800 epoch=8: 5.121973
2016-12-08 22:37:01: Loss after num_examples_seen=900 epoch=9: 5.037522


In [110]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [word_to_index[sentence_start_token]]
    # Repeat until we get an end token
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs = model.forward_propagation(new_sentence)[0]
        sampled_word = word_to_index[unknown_token]
        # We don't want to sample unknown words
        while sampled_word == word_to_index[unknown_token]:
            #print type(next_word_probs[-1])
            #print next_word_probs[-1].shape
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    #print str(len(index_to_word)) + ' index_to_word'
    #for x in new_sentence[1:-1]:
        #print x
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str
 
num_sentences = 10
senten_min_length = 7
 
for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two words
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)

program at Did this debt you to countries I 's that -- sights now tax good they class .
situation Putin you from our , exact be them largely have and , at the .
look Old you to exact world forward say our Secretary this you look is you it because I 're at all this luck do so just , to you this , just .
hired they rebuild -- they are much a you .
clean new do tax decided me mean years 're do fine this .
run only we because biggest just do disgrace .
rights won , fact them country our are I Clinton I .
appointing been 800 years way I 're to same refuses you doing some for to debt , in me , , of this 're , you parts doing he the Instead and big I look love , 28 when 're you years I Democrat , .
How want 20 to 're fast a to n't fighting the great are .
end 30,000 I you years underleveraged companies crazy energy button their front States Kenya , 1,800 end endorsing look where do out of , at these say better .


In [9]:
sentences_df.head(n=5)

,Line,Speaker,Text,Date
0,0.0,Clinton,"How are you, Donald?",9/26/16
1,1.0,Clinton,"Well, thank you, Lester, and thanks to Hofstra...",9/26/16
2,2.0,Clinton,The central question in this election is reall...,9/26/16
3,3.0,Clinton,"Today is my granddaughter's second birthday, s...",9/26/16
4,4.0,Clinton,"First, we have to build an economy that works ...",9/26/16


In [18]:
x_text = []
y_cand = []
for i in range(len(df['Speaker'].values)):
    if df['Speaker'].values[i] == 'Trump':
        x_text.append(df['Text'].values[i])
        y_cand.append('Trump')
    elif df['Speaker'].values[i] == 'Clinton':
        x_text.append(df['Text'].values[i])
        y_cand.append('Clinton')
        

In [3]:
c_text = []
t_text = []
for i in range(len(df['Speaker'].values)):
    if df['Speaker'].values[i] == 'Trump':
        t_text.append(df['Text'].values[i])
    elif df['Speaker'].values[i] == 'Clinton':
        c_text.append(df['Text'].values[i])
        

In [5]:
name = 'clintonwords.txt'
f = open(name,'w')
for i in range(len(c_text)):
	f.write(c_text[i] + ' ')
f.close()

In [6]:
name = 'trumpwords.txt'
f = open(name,'w')
for i in range(len(t_text)):
	f.write(t_text[i] + ' ')
f.close()

In [34]:
vectorizer = CountVectorizer(encoding='latin1', stop_words=['and', 'or', 'before', 'a', 'an', 'am', 'the', 'at', 'by', 'br'], min_df=4)
x = vectorizer.fit_transform(x_text)
x = x.toarray()
print type(x)

<type 'numpy.ndarray'>


In [26]:
log = LogReg()
log.fit(x, y_cand)
log.score(x, y_cand)

0.95986622073578598

In [37]:
print len(x)
print len(x[0])
print len(vectorizer.vocabulary_)
print len(x_text)

598
982
982
598


In [41]:
word_counts = zip(x_text, x)

In [ ]:
#features = []
for i in range(len(x_text)):
    if 